In [1]:
import sys, os
sys.path.append(os.path.abspath('../'))

In [2]:
# saved files testing
from src.utils.flow import *
from src.utils.flow_calculations import *
from src.utils.restoration import *
from src.utils.truncated_packet import *
from src.operations.size_perturbation_logic import *
from src.operations.timing_perturbation_logic import *

modified_pcap_path = "../data/interim/testing_small_perturbed.pcapng"
pcap_file_path = '../data/raw/botnet-capture-20110816-donbot.pcap'

In [31]:
truncated_packets = create_truncated_packets_from_pcap(pcap_file_path)
truncated_packets = assign_flow_ids_to_packets(truncated_packets)
flow_id = 1

### operations

1) size - infuences timing results so has to be first
   - uniform 
     - total scaling (+/-) (*)
   - normal
     - normal scaling with mean&std (+)

  
2) time - doesn't influence size results
   - uniform
     - total scaling (+/-) (*)
   - normal
     - normal scaling with mean&std (+)

Optimization phases:
1) optimize size with sequence of steps to minimize fitness error, direction should be taken into consideration, output should be the best possible obtained result for each flow; 
        fitness attributes: "tot_fwd_pkts", "tot_bwd_pkts", "totlen_fwd_pkts", "totlen_bwd_pkts",
        "fwd_pkt_len_max", "fwd_pkt_len_min", "fwd_pkt_len_mean", "fwd_pkt_len_std",
        "bwd_pkt_len_max", "bwd_pkt_len_min", "bwd_pkt_len_mean", "bwd_pkt_len_std",
        "pkt_len_mean", "pkt_len_std",
2) optimize timestamp with sequence of steps to minimize fitness error, directoin can't be taken into consideration dute to timing constrains, optimizing each direction independently would mess up the order; 
        fitness attributes: "flow_iat_mean", "flow_iat_max", "flow_iat_min", "flow_iat_std",
        "fwd_iat_tot", "fwd_iat_max", "fwd_iat_min", "fwd_iat_mean", "fwd_iat_std",
        "bwd_iat_tot", "bwd_iat_max", "bwd_iat_min", "bwd_iat_mean", "bwd_iat_std",

Operation space:
Size - normal:
adjust_packet_size(truncated_packets, flow_id, direction, method='uniform', scaling_factor, focus_point)
pocus_point={"start", "middle", "end"}
scaling_factor = [1.1, 5.0]

Size - uniform:
adjust_packet_size(truncated_packets, flow_id, method="uniform", scaling_factor, direction)
scaling_factor = [0.25, 10.0]
direction = {0, 1, 2}

Time - normal:
apply_time_perturbation_with_focus(truncated_packets, flow_id, focus_point, method="normal", scaling_factor)
pocus_point={"start", "middle", "end"}
scaling_factor=[1.1, 3.0]

Time - uniform:
apply_time_perturbation_with_focus(truncated_packets, flow_id, method="uniform", scaling_factor)
scaling_factor=[0.4, 2.4]


Fitness function:
predict_single_flow(model_name, sizing_stats, timing_stats)


In [4]:
# target dataset import
import pandas as pd

target_dataset = pd.read_csv('../data/processed/CTU13_Normal_Traffic_20.csv')
additional_test = pd.read_csv('../data/processed/CTU13_Normal_Traffic_80.csv')

In [5]:
# TODO: standaryzacja, tam w ogole nie wykonuje sie taka loguka tego bledu jaka miala
# TODO: weryfikacja bo coś tam się chyba nie zgadza a to bardzo wazne

In [6]:
# sizing

def prepare_size_stats(truncated_packets, flow_id):
    
    size_stats = calculate_size_statistics(flow_id, truncated_packets, direction=0)
    size_stats_fwd = calculate_size_statistics(flow_id, truncated_packets, direction=1)
    size_stats_bwd = calculate_size_statistics(flow_id, truncated_packets, direction=2)
    
    total_fwd_pkts = total_packet_count(flow_id, truncated_packets, direction=1)
    total_bwd_pkts = total_packet_count(flow_id, truncated_packets, direction=2)
    
    totlen_fwd_pkts = total_flow_size(flow_id, truncated_packets, direction=1)
    totlen_bwd_pkts = total_flow_size(flow_id, truncated_packets, direction=2)
    
    

    adjusted_size_stats = {
        "tot_fwd_pkts": total_fwd_pkts,
        "tot_bwd_pkts": total_bwd_pkts,
        "totlen_fwd_pkts": totlen_fwd_pkts,
        "totlen_bwd_pkts": totlen_bwd_pkts,
        "fwd_pkt_len_max": size_stats_fwd['max'],
        "fwd_pkt_len_min": size_stats_fwd['min'],
        "fwd_pkt_len_mean": size_stats_fwd['mean'],
        "fwd_pkt_len_std": size_stats_fwd['std'],
        "bwd_pkt_len_max": size_stats_bwd['max'],
        "bwd_pkt_len_min": size_stats_bwd['min'],
        "bwd_pkt_len_mean": size_stats_bwd['mean'],
        "bwd_pkt_len_std": size_stats_bwd['std'],
        "pkt_len_mean": size_stats['mean'],
        "pkt_len_std": size_stats['std'],
    }
    
    

    return adjusted_size_stats



In [7]:
prepare_size_stats(truncated_packets, flow_id)

{'tot_fwd_pkts': 1090,
 'tot_bwd_pkts': 635,
 'totlen_fwd_pkts': 65646,
 'totlen_bwd_pkts': 1209780,
 'fwd_pkt_len_max': 174,
 'fwd_pkt_len_min': 60,
 'fwd_pkt_len_mean': 60.225688073394494,
 'fwd_pkt_len_std': 4.885710369123273,
 'bwd_pkt_len_max': 4434,
 'bwd_pkt_len_min': 60,
 'bwd_pkt_len_mean': 1905.1653543307086,
 'bwd_pkt_len_std': 701.5260190007999,
 'pkt_len_mean': 739.3773913043478,
 'pkt_len_std': 986.3701563824255}

In [36]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
import numpy as np

def calculate_average_mse(flow_stats, target_dataset):
    """
    Calculate the average Mean Squared Error (MSE) between a single record (dict or pd.Series) and 
    each record in a target dataset, after standardizing the datasets to ensure each attribute 
    contributes equally.
    
    Parameters:
    - flow_stats (dict or pd.Series): A single record containing the attributes.
    - target_dataset (pd.DataFrame): A pandas DataFrame containing the target dataset records.
    
    Returns:
    - float: The average MSE across all records in the target dataset compared to the single record.
    """
    
    # Check if flow_stats is a dict and convert to pd.DataFrame, else create DataFrame directly
    if isinstance(flow_stats, dict):
        single_record_df = pd.DataFrame([flow_stats])
    elif isinstance(flow_stats, pd.Series):
        single_record_df = pd.DataFrame([flow_stats.to_dict()])
    else:
        raise ValueError("flow_stats must be either a dict or a pd.Series")

    # Filter target_dataset to match flow_stats keys
    filtered_target_dataset = target_dataset[single_record_df.columns]

    # Standardize both datasets
    scaler = StandardScaler()
    scaler.fit(filtered_target_dataset)
    standardized_single_record = scaler.transform(single_record_df)
    standardized_target_dataset = scaler.transform(filtered_target_dataset)


    # Calculate MSE between the single record and each record in the target dataset
    mse = np.mean((standardized_target_dataset - standardized_single_record) ** 2, axis=1)
    print(mse)

    # Calculate the average MSE across all comparisons
    average_mse = np.median(mse) # median to minimize outliers bias

    return average_mse


In [44]:
import pandas as pd
import numpy as np

def calculate_average_mse_without_scaling(flow_stats, target_dataset):
    """
    Calculate the average Mean Squared Error (MSE) between a single record (dict or pd.Series) and 
    each record in a target dataset without standardizing the datasets.
    
    Parameters:
    - flow_stats (dict or pd.Series): A single record containing the attributes.
    - target_dataset (pd.DataFrame): A pandas DataFrame containing the target dataset records.
    
    Returns:
    - float: The average MSE across all records in the target dataset compared to the single record.
    """
    
    if isinstance(flow_stats, dict):
        single_record_df = pd.DataFrame([flow_stats])
    elif isinstance(flow_stats, pd.Series):
        single_record_df = pd.DataFrame([flow_stats.to_dict()])
    else:
        raise ValueError("flow_stats must be either a dict or a pd.Series")

    # Ensure target_dataset only contains columns present in flow_stats
    filtered_target_dataset = target_dataset[list(single_record_df.columns)]

    # Direct comparison without scaling
    mse = np.mean((filtered_target_dataset.values - single_record_df.values) ** 2, axis=1)
    print(mse)

    # Calculate the average MSE across all comparisons
    average_mse = np.median(mse)  # Using median to minimize outliers bias

    return average_mse


In [9]:
target_dataset.head()

,tot_fwd_pkts,tot_bwd_pkts,totlen_fwd_pkts,totlen_bwd_pkts,fwd_pkt_len_max,fwd_pkt_len_min,fwd_pkt_len_mean,fwd_pkt_len_std,bwd_pkt_len_max,bwd_pkt_len_min,...,fwd_iat_std,bwd_iat_tot,bwd_iat_max,bwd_iat_min,bwd_iat_mean,bwd_iat_std,fin_flag_cnt,syn_flag_cnt,rst_flag_cnt,ack_flag_cnt
0,0,2,0,160,0,0,0.0,0.0,118,42,...,0.0,361782,361782,361782,361782.0,0.0,0,0,0,0
1,0,2,0,172,0,0,0.0,0.0,116,56,...,0.0,53234,53234,53234,53234.0,0.0,0,0,0,0
2,2,2,0,0,0,0,0.0,0.0,0,0,...,0.0,9744,9744,9744,9744.0,0.0,0,1,0,0
3,0,2,0,82,0,0,0.0,0.0,49,33,...,0.0,36833,36833,36833,36833.0,0.0,0,0,0,0
4,0,2,0,224,0,0,0.0,0.0,175,49,...,0.0,46929,46929,46929,46929.0,0.0,0,0,0,0


In [10]:
adjusted_size_stats = prepare_size_stats(truncated_packets, 1)
mad_sizing = calculate_average_mse(adjusted_size_stats, target_dataset)
adjusted_size_stats

[26.27952437 26.219086   28.79536913 ... 28.80154497 26.47745213
 27.47306939]


{'tot_fwd_pkts': 1090,
 'tot_bwd_pkts': 635,
 'totlen_fwd_pkts': 65646,
 'totlen_bwd_pkts': 1209780,
 'fwd_pkt_len_max': 174,
 'fwd_pkt_len_min': 60,
 'fwd_pkt_len_mean': 60.225688073394494,
 'fwd_pkt_len_std': 4.885710369123273,
 'bwd_pkt_len_max': 4434,
 'bwd_pkt_len_min': 60,
 'bwd_pkt_len_mean': 1905.1653543307086,
 'bwd_pkt_len_std': 701.5260190007999,
 'pkt_len_mean': 739.3773913043478,
 'pkt_len_std': 986.3701563824255}

In [11]:
mad_sizing

26.541310671868793

In [12]:
truncated_packets

[TruncatedPacket(packet_id=1, timestamp=1313482194.088306, size=62, pseudo_hash='-559616267432207075', flow_id=1, direction='1', src_ip=147.32.84.165, fragmented=0, tcp=1, udp=0, fin=0, syn=1, rst=0, ack=0, psh=0, urg=0),
 TruncatedPacket(packet_id=2, timestamp=1313482194.088315, size=62, pseudo_hash='-559616267432207075', flow_id=1, direction='1', src_ip=147.32.84.165, fragmented=0, tcp=1, udp=0, fin=0, syn=1, rst=0, ack=0, psh=0, urg=0),
 TruncatedPacket(packet_id=3, timestamp=1313482194.151276, size=62, pseudo_hash='-559616267432207075', flow_id=1, direction='2', src_ip=91.212.135.158, fragmented=0, tcp=1, udp=0, fin=0, syn=1, rst=0, ack=1, psh=0, urg=0),
 TruncatedPacket(packet_id=4, timestamp=1313482194.151598, size=60, pseudo_hash='-559616267432207075', flow_id=1, direction='1', src_ip=147.32.84.165, fragmented=0, tcp=1, udp=0, fin=0, syn=0, rst=0, ack=1, psh=0, urg=0),
 TruncatedPacket(packet_id=5, timestamp=1313482194.151610, size=60, pseudo_hash='-559616267432207075', flow_id=

In [49]:
sum = 0.0

for i in range(1, 2000):
    #truncated_packets = adjust_packet_size(truncated_packets, i, method="normal", scaling_factor=1000.0, direction= 1, focus_point="start")
    #truncated_packets = adjust_packet_size(truncated_packets, i, method="uniform", scaling_factor=0.01, direction = 0)
    adjusted_size_stats = prepare_size_stats(truncated_packets, i)
    mad_sizing = calculate_average_mse_without_scaling(adjusted_size_stats, target_dataset)
    sum = sum + mad_sizing
    print(f'mse:{mad_sizing}, fwd_packets: {adjusted_size_stats["tot_fwd_pkts"]}, bwd_packets: {adjusted_size_stats["tot_bwd_pkts"]}')
    
print(sum/2000)
    

[1.04822521e+11 1.04820448e+11 1.04850284e+11 ... 1.04850284e+11
 1.04824262e+11 1.04836064e+11]
mse:104824433248.61015, fwd_packets: 1090, bwd_packets: 635
[ 5238.53103678  4915.88289869 10394.47619048 ... 10394.76190476
  5153.71292767  6623.69575715]
mse:7034.281639455533, fwd_packets: 2, bwd_packets: 1
[10462.69320718  9739.38436153 21878.83333333 ... 21879.11904762
 10817.52540411 14972.52313138]
mse:13451.380274230874, fwd_packets: 2, bwd_packets: 1
[ 4363.98267408  4023.46501837 10044.71428571 ... 10045.
  4327.06576319  6027.70813191]
mse:6103.231941438914, fwd_packets: 2, bwd_packets: 1
[2062148.00588941 2057472.56698425 2230862.41632653 ... 2230866.13061224
 2077847.73503515 2151639.84094625]
mse:2083869.1027493847, fwd_packets: 14, bwd_packets: 6
[8970.41269842 9273.14285713 5492.         ... 5492.85714286 8330.41269841
 6123.17460319]
mse:8299.428571414524, fwd_packets: 4, bwd_packets: 0
[8970.41269842 9273.14285713 5492.         ... 5492.85714286 8330.41269841
 6123.174603

In [14]:
truncated_packets

[TruncatedPacket(packet_id=1, timestamp=1313482194.088306, size=62, pseudo_hash='-559616267432207075', flow_id=1, direction='1', src_ip=147.32.84.165, fragmented=0, tcp=1, udp=0, fin=0, syn=1, rst=0, ack=0, psh=0, urg=0),
 TruncatedPacket(packet_id=2, timestamp=1313482194.088315, size=62, pseudo_hash='-559616267432207075', flow_id=1, direction='1', src_ip=147.32.84.165, fragmented=0, tcp=1, udp=0, fin=0, syn=1, rst=0, ack=0, psh=0, urg=0),
 TruncatedPacket(packet_id=3, timestamp=1313482194.151276, size=62, pseudo_hash='-559616267432207075', flow_id=1, direction='2', src_ip=91.212.135.158, fragmented=0, tcp=1, udp=0, fin=0, syn=1, rst=0, ack=1, psh=0, urg=0),
 TruncatedPacket(packet_id=4, timestamp=1313482194.151598, size=60, pseudo_hash='-559616267432207075', flow_id=1, direction='1', src_ip=147.32.84.165, fragmented=0, tcp=1, udp=0, fin=0, syn=0, rst=0, ack=1, psh=0, urg=0),
 TruncatedPacket(packet_id=5, timestamp=1313482194.151610, size=60, pseudo_hash='-559616267432207075', flow_id=

In [50]:
sum = 0
iter = 0
cols_to_drop = ["flow_iat_mean", "flow_iat_max", "flow_iat_min", "flow_iat_std",
 "fwd_iat_tot", "fwd_iat_max", "fwd_iat_min", "fwd_iat_mean", "fwd_iat_std",
 "bwd_iat_tot", "bwd_iat_max", "bwd_iat_min", "bwd_iat_mean", "bwd_iat_std", "tot_fwd_pkts", "tot_bwd_pkts", "totlen_fwd_pkts", "totlen_bwd_pkts"]

for index, row in additional_test.iterrows():
    # Obliczenie 'mad_sizing' dla aktualnego wiersza
    mad_sizing = calculate_average_mse_without_scaling(row.drop(cols_to_drop), target_dataset)
    iter = iter + 1
    # Pobranie wartości dla 'tot_fwd_pkts' i 'tot_bwd_pkts' z aktualnego wiersza
    tot_fwd_pkts = row['tot_fwd_pkts']
    tot_bwd_pkts = row['tot_bwd_pkts']
    tot_pkts = tot_fwd_pkts + tot_bwd_pkts
    
    sum = sum + mad_sizing
    # Wypisanie wyników
    print(f"Record {index}: MAD Sizing = {mad_sizing:.4f}, FWD Packets = {tot_fwd_pkts}, BWD Packets = {tot_bwd_pkts}, TOT Packets = {tot_pkts}")
    if (iter > 2000):
        break
sum / 2000


[  12.79960317    8.75198413 2182.49007943 ... 2182.49007943   18.55357143
  654.82341275]
Record 0: MAD Sizing = 1322.5615, FWD Packets = 0.0, BWD Packets = 2.0, TOT Packets = 2.0
[  23.99007937   51.84722223 2732.72817463 ... 2732.72817463  114.26785714
 1005.2996032 ]
Record 1: MAD Sizing = 1124.4425, FWD Packets = 0.0, BWD Packets = 2.0, TOT Packets = 2.0
[  69.12500001  109.20436509 1528.75198408 ... 1528.75198408   24.08531747
  355.3392857 ]
Record 2: MAD Sizing = 1524.7312, FWD Packets = 0.0, BWD Packets = 2.0, TOT Packets = 2.0
[2.24550794e+03 2.34042857e+03 2.14285714e-01 ... 2.14285714e-01
 1.81217460e+03 4.89698413e+02]
Record 3: MAD Sizing = 1850.1429, FWD Packets = 1.0, BWD Packets = 1.0, TOT Packets = 2.0
[  39.48214286   60.5138889  1718.63293646 ... 1718.63293646    6.34722223
  429.98214284]
Record 4: MAD Sizing = 1612.4187, FWD Packets = 0.0, BWD Packets = 2.0, TOT Packets = 2.0
[2.24543651e+03 2.34035714e+03 1.42857143e-01 ... 1.42857143e-01
 1.81210317e+03 4.896269

26525.766525668463

In [48]:
sum = 0
iter = 0
cols_to_drop = ["flow_iat_mean", "flow_iat_max", "flow_iat_min", "flow_iat_std",
 "fwd_iat_tot", "fwd_iat_max", "fwd_iat_min", "fwd_iat_mean", "fwd_iat_std",
 "bwd_iat_tot", "bwd_iat_max", "bwd_iat_min", "bwd_iat_mean", "bwd_iat_std", "tot_fwd_pkts", "tot_bwd_pkts", "totlen_fwd_pkts", "totlen_bwd_pkts"]

for index, row in target_dataset.iterrows():
    # Obliczenie 'mad_sizing' dla aktualnego wiersza
    mad_sizing = calculate_average_mse(row.drop(cols_to_drop), target_dataset)
    iter = iter + 1
    # Pobranie wartości dla 'tot_fwd_pkts' i 'tot_bwd_pkts' z aktualnego wiersza
    tot_fwd_pkts = row['tot_fwd_pkts']
    tot_bwd_pkts = row['tot_bwd_pkts']
    tot_pkts = tot_fwd_pkts + tot_bwd_pkts
    
    sum = sum + mad_sizing
    # Wypisanie wyników
    print(f"Record {index}: MAD Sizing = {mad_sizing:.4f}, FWD Packets = {tot_fwd_pkts}, BWD Packets = {tot_bwd_pkts}, TOT Packets = {tot_pkts}")
    if (iter > 2000):
        break
sum / 2000


[0.         0.02828614 0.84281967 ... 3.85280086 0.00452363 0.02905789]
Record 0: MAD Sizing = 0.0839, FWD Packets = 0.0, BWD Packets = 2.0, TOT Packets = 2.0
[0.02828614 0.         1.0372958  ... 4.047277   0.01219907 0.09171284]
Record 1: MAD Sizing = 0.1511, FWD Packets = 0.0, BWD Packets = 2.0, TOT Packets = 2.0
[0.84281967 1.0372958  0.         ... 4.09151404 0.89453829 0.70522091]
Record 2: MAD Sizing = 0.9474, FWD Packets = 2.0, BWD Packets = 2.0, TOT Packets = 4.0
[0.02905789 0.09171284 0.70522091 ... 3.71520211 0.03853465 0.        ]
Record 3: MAD Sizing = 0.1619, FWD Packets = 0.0, BWD Packets = 2.0, TOT Packets = 2.0
[0.01894889 0.02059854 0.9942274  ... 4.0042086  0.02001357 0.09491629]
Record 4: MAD Sizing = 0.0964, FWD Packets = 0.0, BWD Packets = 2.0, TOT Packets = 2.0
[0.71268036 0.90739231 0.95905207 ... 3.96903327 0.76544702 0.57967201]
Record 5: MAD Sizing = 0.7195, FWD Packets = 2.0, BWD Packets = 2.0, TOT Packets = 4.0
[0.70870595 0.90318208 0.94741913 ... 3.957400

1.0923691835029559

In [28]:
# timing
def prepare_timing_stats(truncated_packets, flow_id):
    """
    Obtain and adjust timing statistics for a specific flow ID from truncated packets,
    considering the entire flow as well as forward and backward directions separately.
    """
    # Obtain timing stats for the entire flow (direction=0)
    timing_stats_total = calculate_delta_time_statistics(flow_id, truncated_packets, direction=0)
    # Obtain timing stats for the forward direction (fwd, direction=1)
    timing_stats_fwd = calculate_delta_time_statistics(flow_id, truncated_packets, direction=1)
    # Obtain timing stats for the backward direction (bwd, direction=2)
    timing_stats_bwd = calculate_delta_time_statistics(flow_id, truncated_packets, direction=2)

    # Combining and adjusting timing stats to the expected format for MAD calculation
    adjusted_timing_stats = {
        "flow_iat_mean": timing_stats_total.get('mean', 0),
        "flow_iat_max": timing_stats_total.get('max', 0),
        "flow_iat_min": timing_stats_total.get('min', 0),
        "flow_iat_std": timing_stats_total.get('std', 0),
        "fwd_iat_tot": timing_stats_fwd.get('total', 0),
        "fwd_iat_max": timing_stats_fwd.get('max', 0),
        "fwd_iat_min": timing_stats_fwd.get('min', 0),
        "fwd_iat_mean": timing_stats_fwd.get('mean', 0),
        "fwd_iat_std": timing_stats_fwd.get('std', 0),
        "bwd_iat_tot": timing_stats_bwd.get('total', 0),
        "bwd_iat_max": timing_stats_bwd.get('max', 0),
        "bwd_iat_min": timing_stats_bwd.get('min', 0),
        "bwd_iat_mean": timing_stats_bwd.get('mean', 0),
        "bwd_iat_std": timing_stats_bwd.get('std', 0),
    }

    return adjusted_timing_stats


In [37]:
timing_stats = prepare_timing_stats(truncated_packets, 1)
print(timing_stats)
mad_timing = calculate_average_mse(timing_stats, target_dataset)

{'flow_iat_mean': 1099.9669373549882, 'flow_iat_max': 163625.0, 'flow_iat_min': 0.0, 'flow_iat_std': 8067.8267672643215, 'fwd_iat_tot': 0, 'fwd_iat_max': 163966.0, 'fwd_iat_min': 0.0, 'fwd_iat_mean': 1683.2203856749313, 'fwd_iat_std': 9974.730315269562, 'bwd_iat_tot': 0, 'bwd_iat_max': 164537.0, 'bwd_iat_min': 5.0, 'bwd_iat_mean': 2891.7555205047315, 'bwd_iat_std': 13004.028339654073}
[8.56282432e-03 2.04369414e-04 1.04706682e+00 ... 3.19720182e-05
 9.37576575e-05 1.13597759e-04]


In [38]:
mad_timing

0.00012323489093711413

In [39]:
sum = 0
iter = 0
for index, row in additional_test.iterrows():
    # Obliczenie 'mad_sizing' dla aktualnego wiersza
    mad_timing = calculate_average_mse(row, target_dataset)
    iter = iter + 1
    # Pobranie wartości dla 'tot_fwd_pkts' i 'tot_bwd_pkts' z aktualnego wiersza
    tot_fwd_pkts = row['tot_fwd_pkts']
    tot_bwd_pkts = row['tot_bwd_pkts']
    tot_pkts = tot_fwd_pkts + tot_bwd_pkts
    
    sum = sum + mad_sizing
    # Wypisanie wyników
    print(f"Record {index}: MAD Sizing = {mad_sizing:.4f}, FWD Packets = {tot_fwd_pkts}, BWD Packets = {tot_bwd_pkts}, TOT Packets = {tot_pkts}")
    if (iter > 2000):
        break
sum / 2000


[6.12035754e-03 3.09529412e-03 8.65522570e-01 ... 1.72400564e+00
 4.44301926e-04 2.26858456e-02]
Record 0: MAD Sizing = 0.0766, FWD Packets = 0.0, BWD Packets = 2.0, TOT Packets = 2.0
[5.14666379e-03 5.51365341e-03 8.59883756e-01 ... 1.71810083e+00
 1.20849355e-03 2.27307620e-02]
Record 1: MAD Sizing = 0.0766, FWD Packets = 0.0, BWD Packets = 2.0, TOT Packets = 2.0
[0.00589101 0.02565961 0.79077446 ... 1.64929345 0.00766309 0.00431422]
Record 2: MAD Sizing = 0.0766, FWD Packets = 0.0, BWD Packets = 2.0, TOT Packets = 2.0
[2.61231399 2.69370558 3.17190685 ... 4.02984483 2.63119845 2.54838129]
Record 3: MAD Sizing = 0.0766, FWD Packets = 1.0, BWD Packets = 1.0, TOT Packets = 2.0
[0.00310841 0.01442276 0.81620673 ... 1.67497198 0.00229011 0.00851329]
Record 4: MAD Sizing = 0.0766, FWD Packets = 0.0, BWD Packets = 2.0, TOT Packets = 2.0
[0.31382973 0.39522406 0.87341885 ... 1.73136377 0.33271711 0.24989991]
Record 5: MAD Sizing = 0.0766, FWD Packets = 1.0, BWD Packets = 1.0, TOT Packets = 

0.0766587759630189

In [40]:
calculate_delta_time_statistics(1, truncated_packets, direction=0)

{'mean': 1099.9669373549882,
 'min': 0.0,
 'max': 163625.0,
 'std': 8067.8267672643215}

In [41]:
sum = 0.0

for i in range(1, 2000):
    #truncated_packets = adjust_packet_size(truncated_packets, i, method="normal", scaling_factor=1000.0, direction= 1, focus_point="start")
    #truncated_packets = adjust_packet_size(truncated_packets, i, method="uniform", scaling_factor=0.01, direction = 0)
    adjusted_size_stats = prepare_timing_stats(truncated_packets, i)
    mad_timing = calculate_average_mse(adjusted_size_stats, target_dataset)
    sum = sum + mad_timing
    print(f'mse:{mad_timing}')
    
    adjusted_size_stats 
    
print(sum/2000)

[8.56282432e-03 2.04369414e-04 1.04706682e+00 ... 3.19720182e-05
 9.37576575e-05 1.13597759e-04]
mse:0.00012323489093711413
[8.30764105e-03 1.91333591e-04 1.04468964e+00 ... 5.91279874e-05
 9.68266469e-05 1.13077873e-04]
mse:0.00012025861517252203
[8.62940575e-03 1.86651289e-04 1.04897644e+00 ... 2.61796379e-07
 7.04329122e-05 9.15224200e-05]
mse:0.0001033492786189301
[8.62027354e-03 1.85346419e-04 1.04887259e+00 ... 2.56113714e-07
 6.96486256e-05 9.06221241e-05]
mse:0.00010214429380842364
[8.28046218e-03 1.99463684e-04 1.04410465e+00 ... 7.31186848e-05
 1.07305079e-04 1.23032989e-04]
mse:0.0001303467298110772
[0.03493287 0.03113887 0.9211447  ... 0.03172409 0.03133183 0.03128402]
mse:0.031398630761560006
[0.03493222 0.03113817 0.92114582 ... 0.03172338 0.03133113 0.03128332]
mse:0.03139792537071138
[0.03493022 0.03113601 0.92114919 ... 0.03172119 0.03132896 0.03128115]
mse:0.03139574796796968
[0.03492793 0.03113351 0.92115354 ... 0.03171866 0.03132645 0.03127864]
mse:0.031393235607950

In [45]:
sum = 0
iter = 0
cols_to_drop = ["tot_fwd_pkts", "tot_bwd_pkts", "totlen_fwd_pkts", "totlen_bwd_pkts",
 "fwd_pkt_len_max", "fwd_pkt_len_min", "fwd_pkt_len_mean", "fwd_pkt_len_std",
 "bwd_pkt_len_max", "bwd_pkt_len_min", "bwd_pkt_len_mean", "bwd_pkt_len_std",
 "pkt_len_mean", "pkt_len_std"]


for index, row in additional_test.iterrows():
    # Obliczenie 'mad_sizing' dla aktualnego wiersza
    mad_timing = calculate_average_mse_without_scaling(row.drop(cols_to_drop), target_dataset)
    iter = iter + 1

    
    sum = sum + mad_timing
    # Wypisanie wyników
    print(f"Record {index}: MAD Timing = {mad_timing:.4f}")
    if (iter > 2000):
        break
sum / 2000


[4.03456018e+10 7.13798960e+07 1.16155928e+13 ... 5.51698088e+08
 1.89088391e+07 2.24000000e+06]
Record 0: MAD Timing = 401826713.6000
[4.52607117e+10 4.13396984e+08 1.16327634e+13 ... 1.34670232e+08
 5.67773457e+07 1.07886464e+08]
Record 1: MAD Timing = 207568903.5000
[3.97033422e+10 4.68333740e+07 1.16132951e+13 ... 6.29679992e+08
 3.54453591e+07 9.62137400e+06]
Record 2: MAD Timing = 424059943.7778
[5.08436730e+10 1.09377059e+09 1.16512392e+13 ... 1.30910439e+06
 4.11088227e+08 5.34858758e+08]
Record 3: MAD Timing = 655657904.3333
[3.54402760e+10 1.72856791e+07 1.15976866e+13 ... 1.30281449e+09
 2.87461745e+08 1.98892643e+08]
Record 4: MAD Timing = 800086486.2222
[5.07825337e+10 1.08487873e+09 1.16507718e+13 ... 1.08930967e+06
 4.05671254e+08 5.28667424e+08]
Record 5: MAD Timing = 648793469.6111
[4.72735291e+13 4.78003691e+13 1.23021372e+13 ... 4.78949794e+13
 4.78380344e+13 4.78296126e+13]
Record 6: MAD Timing = 47838917531414.2812
[1.83746330e+13 1.91845392e+13 1.31798069e+13 ... 

142832433479703.22

In [46]:
sum = 0
iter = 0
cols_to_drop = ["tot_fwd_pkts", "tot_bwd_pkts", "totlen_fwd_pkts", "totlen_bwd_pkts",
 "fwd_pkt_len_max", "fwd_pkt_len_min", "fwd_pkt_len_mean", "fwd_pkt_len_std",
 "bwd_pkt_len_max", "bwd_pkt_len_min", "bwd_pkt_len_mean", "bwd_pkt_len_std",
 "pkt_len_mean", "pkt_len_std"]


for index, row in target_dataset.iterrows():
    # Obliczenie 'mad_sizing' dla aktualnego wiersza
    mad_timing = calculate_average_mse_without_scaling(row.drop(cols_to_drop), target_dataset)
    iter = iter + 1

    
    sum = sum + mad_timing
    # Wypisanie wyników
    print(f"Record {index}: MAD Timing = {mad_timing:.4f}")
    if (iter > 2000):
        break
sum / 2000


[0.00000000e+00 3.70229488e+10 1.13680971e+13 ... 5.03331022e+10
 4.21113804e+10 4.09490877e+10]
Record 0: MAD Timing = 44588465787.0556
[3.70229488e+10 0.00000000e+00 1.16035570e+13 ... 1.01996664e+09
 1.63765560e+08 9.89094960e+07]
Record 1: MAD Timing = 629746698.4444
[1.13680971e+13 1.16035570e+13 0.00000000e+00 ... 1.16498038e+13
 1.16218431e+13 1.16177398e+13]
Record 2: MAD Timing = 11623357372755.4023
[4.10634982e+10 1.04608312e+08 1.16181455e+13 ... 4.71284856e+08
 6.60115556e+06 7.98035000e+04]
Record 3: MAD Timing = 369213077.7778
[3.85514934e+10 1.54595097e+07 1.16091405e+13 ... 7.84283292e+08
 7.85923662e+07 3.61617857e+07]
Record 4: MAD Timing = 495511348.7222
[2.96257750e+10 1.62895228e+09 1.15332182e+13 ... 4.14708126e+09
 2.39303005e+09 2.19439748e+09]
Record 5: MAD Timing = 3046372636.2515
[4.95307354e+10 9.19477996e+08 1.16410739e+13 ... 1.60095472e+07
 3.12524959e+08 4.19419596e+08]
Record 6: MAD Timing = 526397373.7778
[4.13211886e+10 1.17997274e+08 1.16190579e+13 .

150574235021381.66